# Рекомендация тарифов

Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами. Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра». Нам нужно решить эту задачу, применив машинное обучение.

Перед нами стоит задача построения модели с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. 

## 1. Откройте и изучите файл

In [1]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
df = pd.read_csv('/datasets/users_behavior.csv')
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
calls       3214 non-null float64
minutes     3214 non-null float64
messages    3214 non-null float64
mb_used     3214 non-null float64
is_ultra    3214 non-null int64
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


Данные подготовлены в предыдущем проекте, никаких манипуляций не будем с ними производить. Расшифруем только значения в датасете: 
сalls — количество звонков,
minutes — суммарная длительность звонков в минутах,
messages — количество sms-сообщений,
mb_used — израсходованный интернет-трафик в Мб,
is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

Таргетом у нас является столбец is_ultra

## 2. Разбейте данные на выборки

Разбиваем датасет на обучающую, тестовую и валидационную выборки в соотношении 60/20/20.

In [4]:
df_train, df_valid_test = train_test_split(df, test_size=0.4, random_state=12345)

In [5]:
df_valid, df_test = train_test_split(df_valid_test, test_size=0.5, random_state=12345)

In [6]:
features_train = df_train.drop(['is_ultra'],axis=1)
target_train = df_train['is_ultra']
features_valid = df_valid.drop(['is_ultra'],axis=1)
target_valid = df_valid['is_ultra']
features_test = df_test.drop(['is_ultra'],axis=1)
target_test = df_test['is_ultra']

## 3. Исследуйте модели

Начнём с дерева принятия решений.

In [7]:
for i in range(1,6):
    model = DecisionTreeClassifier(random_state=12345,max_depth = i)
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid)
    
    print("max_depth =", i, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth = 1 : 0.7542768273716952
max_depth = 2 : 0.7822706065318819
max_depth = 3 : 0.7853810264385692
max_depth = 4 : 0.7791601866251944
max_depth = 5 : 0.7791601866251944


Лучший показатель accuracy (0.785) найден на глубине в значении 3. Теперь поехали смотреть модель случайного леса. Определимся сначала, какая глубина будет в данном случае самой результативной:

In [8]:
for i in range(2,15,1):
    model = RandomForestClassifier(random_state=12345, max_depth=i)
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid)
    
    print("max_depth =", i, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

max_depth = 2 : 0.7776049766718507
max_depth = 3 : 0.7853810264385692


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The de

max_depth = 4 : 0.7900466562986003
max_depth = 5 : 0.7931570762052877
max_depth = 6 : 0.8009331259720062
max_depth = 7 : 0.7947122861586314
max_depth = 8 : 0.7962674961119751


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


max_depth = 9 : 0.7853810264385692
max_depth = 10 : 0.7916018662519441
max_depth = 11 : 0.7962674961119751


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


max_depth = 12 : 0.7884914463452566
max_depth = 13 : 0.7853810264385692
max_depth = 14 : 0.7776049766718507


/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/ensemble/forest.py:245: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


Далеко идти не пришлось, берём значение глубины в 6-ку. Теперь поехали смотреть количество деревьев:

In [9]:
for i in range(10,200,10):
    model = RandomForestClassifier(random_state=12345, n_estimators = i)
    model.fit(features_train,target_train)
    predictions_valid = model.predict(features_valid)
    
    print("n_estimators", i, ": ", end='')
    print(accuracy_score(target_valid, predictions_valid))

n_estimators 10 : 0.7853810264385692
n_estimators 20 : 0.7869362363919129
n_estimators 30 : 0.7838258164852255
n_estimators 40 : 0.7838258164852255
n_estimators 50 : 0.7916018662519441
n_estimators 60 : 0.7853810264385692
n_estimators 70 : 0.7791601866251944
n_estimators 80 : 0.7822706065318819
n_estimators 90 : 0.7838258164852255
n_estimators 100 : 0.7853810264385692
n_estimators 110 : 0.7838258164852255
n_estimators 120 : 0.7853810264385692
n_estimators 130 : 0.7853810264385692
n_estimators 140 : 0.7900466562986003
n_estimators 150 : 0.7869362363919129
n_estimators 160 : 0.7869362363919129
n_estimators 170 : 0.7869362363919129
n_estimators 180 : 0.7884914463452566
n_estimators 190 : 0.7884914463452566


Лучший показатель accuracy (0.796) найден в гиперпараметре в 50 деревеьв. 

Теперь сделаем логистическую регрессию.

In [10]:
model = LogisticRegression(random_state=12345, C=1.0, solver='liblinear')
model.fit(features_train,target_train);
predictions_valid = model.predict(features_valid)
accuracy_score(target_valid, predictions_valid)

0.7589424572317263

In [11]:
from tqdm.notebook import tqdm

In [12]:
for _ in tqdm(range(2000000), desc='1 loop'):
    pass

Можем сделать вывод, что модель случайного леса со значением в 80 деревьев является лучшей, с такими гиперпараметрами проверяем модель на тестовой выборке.

## 4. Проверьте модель на тестовой выборке

In [13]:
model = RandomForestClassifier(random_state=122345, n_estimators=120, max_depth=10)
model.fit(pd.concat([features_train, features_valid]), pd.concat([target_train, target_valid]))
predictions_test = model.predict(features_test)
print("n_estimators = 120", ":", accuracy_score(target_test, predictions_test))

n_estimators = 120 : 0.8133748055987559


In [14]:
model = RandomForestClassifier(random_state=122345, n_estimators=50, max_depth=6)
model.fit(pd.concat([features_train, features_valid]), pd.concat([target_train, target_valid]))
predictions_test = model.predict(features_test)
print("n_estimators = 50", ":", accuracy_score(target_test, predictions_test))

n_estimators = 50 : 0.7931570762052877


<a class="anchor" id="21-bullet">

Получили accuracy score значением выше 0.75, можно считать задачу выполненной.

## 5. (бонус) Проверьте модели на адекватность

In [15]:
CONST = (df['is_ultra'].value_counts() / df.shape[0]).loc[0]
if CONST < 0.79:
    print('Модель адекватна')

Модель адекватна


In [16]:
1 - df['is_ultra'].mean()

0.693528313627878

In [17]:
(df['is_ultra']==0).sum() / df.shape[0]

0.693528313627878

In [19]:
df['is_ultra'].value_counts(normalize=True)

0    0.693528
1    0.306472
Name: is_ultra, dtype: float64